In [1]:
import pandas as pd
from gensim.models import Word2Vec
import nltk
import re
from nltk import word_tokenize, pos_tag
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import enchant
import numpy as np
from collections import Counter
import seaborn as sns


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import random
from sklearn.model_selection import KFold



from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

import emoji
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords



#nltk.download('punkt_tab')
#nltk.download("words")
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger_eng')
#%matplotlib qt
#nltk.download('stopwords')
%matplotlib inline

In [2]:
df = pd.read_csv("scitweets_export.tsv", sep = "\t")
df = df[df["science_related"] == 1]

In [3]:
df

,Unnamed: 0,tweet_id,text,science_related,scientific_claim,scientific_reference,scientific_context
2,2,322030931022065664,Can any Gynecologist with Cancer Experience ex...,1,1.0,0.0,0.0
3,3,322694830620807168,Couch-lock highs lead to sleeping in the couch...,1,1.0,0.0,0.0
4,4,328524426658328576,Does daily routine help prevent problems with ...,1,1.0,0.0,0.0
6,8,333266791960809472,“Traffic Jam” In Brain’s Neurons Could Be Caus...,1,1.0,1.0,1.0
7,9,334282732085587968,Can playing more games improve lives and save ...,1,1.0,0.0,0.0
...,...,...,...,...,...,...,...
1128,1247,1336903053362954241,Three systematic reviews & the WHO contradict ...,1,0.0,1.0,1.0
1130,1249,1338450113765789699,This looks like a great opportunity to get res...,1,0.0,0.0,1.0
1131,1250,1338483958397480972,Highly prestigious and competitive awards fund...,1,0.0,0.0,1.0
1134,1253,1339398314563792900,"Vestislav Apostolov, David M. J. Calderbank, E...",1,0.0,1.0,1.0


In [26]:
claim = 0
ref = 0
claim_ref = []

for i in range(len(df)):
    claim = int(df.iloc[i]["scientific_claim"])
    ref = int(df.iloc[i]["scientific_reference"])
    if claim == 0 and ref == 0:
        claim_ref.append(0)
    else:
        claim_ref.append(1)

df["claim_ref"] = claim_ref

convert = {"scientific_context": int}
df = df.astype(convert)

In [27]:
df

,Unnamed: 0,tweet_id,text,science_related,scientific_claim,scientific_reference,scientific_context,claim_ref
2,2,322030931022065664,Can any Gynecologist with Cancer Experience ex...,1,1.0,0.0,0,1
3,3,322694830620807168,Couch-lock highs lead to sleeping in the couch...,1,1.0,0.0,0,1
4,4,328524426658328576,Does daily routine help prevent problems with ...,1,1.0,0.0,0,1
6,8,333266791960809472,“Traffic Jam” In Brain’s Neurons Could Be Caus...,1,1.0,1.0,1,1
7,9,334282732085587968,Can playing more games improve lives and save ...,1,1.0,0.0,0,1
...,...,...,...,...,...,...,...,...
1128,1247,1336903053362954241,Three systematic reviews & the WHO contradict ...,1,0.0,1.0,1,1
1130,1249,1338450113765789699,This looks like a great opportunity to get res...,1,0.0,0.0,1,0
1131,1250,1338483958397480972,Highly prestigious and competitive awards fund...,1,0.0,0.0,1,0
1134,1253,1339398314563792900,"Vestislav Apostolov, David M. J. Calderbank, E...",1,0.0,1.0,1,1


In [28]:
test_df = pd.DataFrame(df, columns=["text", "claim_ref", "scientific_context"])
test_df

,text,claim_ref,scientific_context
2,Can any Gynecologist with Cancer Experience ex...,1,0
3,Couch-lock highs lead to sleeping in the couch...,1,0
4,Does daily routine help prevent problems with ...,1,0
6,“Traffic Jam” In Brain’s Neurons Could Be Caus...,1,1
7,Can playing more games improve lives and save ...,1,0
...,...,...,...
1128,Three systematic reviews & the WHO contradict ...,1,1
1130,This looks like a great opportunity to get res...,0,1
1131,Highly prestigious and competitive awards fund...,0,1
1134,"Vestislav Apostolov, David M. J. Calderbank, E...",1,1


In [29]:
len(df)

375

In [30]:
print(len(df[df["claim_ref"] == 1]))
print(len(df[df["claim_ref"] == 0]))

342
33


In [31]:
print(len(df[df["scientific_context"] == 1]))
print(len(df[df["scientific_context"] == 0]))

251
124


In [47]:

print("1 & 1:\t ", len(df.loc[(df["claim_ref"] == 1) & (df["scientific_context"] == 1)]))
print("1 & 0:\t", len(df.loc[(df["claim_ref"] == 1) & (df["scientific_context"] == 0)]))
print("0 & 1:\t", len(df.loc[(df["claim_ref"] == 0) & (df["scientific_context"] == 1)]))

1 & 1:	  218
1 & 0:	 124
0 & 1:	 33
